# Import

In [1]:
import os

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

from sklearn import metrics
from scipy import stats
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler

from shapely.geometry import LineString, MultiPolygon, Polygon
from shapely.ops import split
from shapely.affinity import translate
import geopandas

pd.set_option("display.max_colwidth", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.float_format", "{:.2f}".format)

palette = "coolwarm"

import warnings

warnings.filterwarnings("ignore")

# EDA and preprocessing

In [2]:
data = pd.read_csv("data/autodrom_unprocessed_df.csv",
                   sep=";",
                   index_col="Unnamed: 0",
                   encoding="cp1251")
print(data.shape)
data[:3]

(376035, 24)


,curr_date,name,bodyType,brand_name,color,fuelType,productionDate,modelDate,numberOfDoors,vehicleConfiguration,vehicleTransmission,vehicleEngine_name,vehicleEngine_engineDisplacement,vehicleEngine_enginePower,offers_price,offers_priceCurrency,offers_url,card_charact,card_price_disc,card_price_mark,card_repair,card_noDocs,card_probegCat,card_probegCat1
0,2024-03-28,"Daihatsu Coo, 2006",Хэтчбек 5 дв.,Daihatsu,Черный,Бензин,2006,2006.00,5.00,1.5 CX limited,АКПП,3SZ-VE,1.5 LTR,109 N12,615000,RUB,https://auto.drom.ru/nakhodka/daihatsu/coo/53197435.html,"1.5 л (109 л.с.), бензин, АКПП, передний, 240 000",NaN,без_оценки,NaN,NaN,NaN,NaN
1,2024-03-28,"Daihatsu Coo, 2006",Хэтчбек 5 дв.,Daihatsu,Серый,Бензин,2006,2006.00,5.00,1.5 CX limited,АКПП,3SZ-VE,1.5 LTR,109 N12,760000,RUB,https://auto.drom.ru/ussuriisk/daihatsu/coo/816310958.html,"1.5 л (109 л.с.), бензин, АКПП, передний, 180 000",NaN,без_оценки,NaN,NaN,NaN,NaN
2,2024-03-28,"Daihatsu Coo, 2006",Хэтчбек 5 дв.,Daihatsu,Синий,Бензин,2006,2006.00,5.00,1.3 CS,АКПП,K3-VE,1.3 LTR,92 N12,595000,RUB,https://auto.drom.ru/berdsk/daihatsu/coo/619957730.html,"1.3 л (92 л.с.), бензин, АКПП, передний, 180 000",NaN,без_оценки,NaN,NaN,NaN,NaN


In [3]:
# https://rosstat.gov.ru/folder/13877

gos_metrics = pd.read_excel("data/2024_свод по субъектам.xlsx")
print(gos_metrics.shape)
gos_metrics[:3]

(85, 7)


,key_,income_per_capita,income_median,cost_living,population,retiree,retiree_perc
0,Адыгея,41568,29281.90,13290.00,498.00,115.40,23.20
1,Алтайский край,35621,24037.00,13753.00,2130.90,550.90,25.90
2,Амурская область,50331,34565.70,17823.00,756.20,162.20,21.50


## Prepocessing

### General preprocessing

In [4]:
df = data.copy()

# drop low informative columns
df = df.drop(["curr_date", "name", "brand_name", "offers_priceCurrency"], axis=1)

# obtain new features and clean existing columns, fill in duplicated columns
# offers_url
df["region"] = df["offers_url"].str.split("/", expand=True)[3]
df["brand"] = df["offers_url"].str.split("/", expand=True)[4]
df["model"] = df["offers_url"].str.split("/", expand=True)[5]
df["car_id"] = df["offers_url"].str.split("/", expand=True)[6]
df["car_id"] = df["car_id"].str.replace(".html", "").astype("int")

# card_charact
df["card_charact"] = df["card_charact"].fillna("NaN")
df["card_engine"] = df["card_charact"].str.split(", ", expand=True)[0]
# delete errors in card data
df = df.loc[
    ((df["card_engine"].str.contains("л")) | (df["card_engine"].str.contains("л.с.")))
    & ~(df["card_engine"].str.contains("электро"))
]

# driveType
drive_values = ["передний", "задний", "4WD"]
df["card_driveType"] = df["card_charact"].apply(
    lambda x: next((value for value in drive_values if value in x), np.nan)
)

# card_probegCat and card_probegCat1 merged
df.loc[
    (~(df.card_probegCat.isna()) | ~(df.card_probegCat1.isna()))
    & (df.card_probegCat != "без_пробега_по_РФ"),
    "flag_new",
] = "new"
df.loc[(df.flag_new.isna()), "flag_new"] = "old"
df.drop(["card_probegCat", "card_probegCat1"], axis=1, inplace=True)
# df['flag_new'] = df['flag_new'].astype('int')

# card_distance_km
df["card_charact"] = df["card_charact"].replace(", \\xa0\w+", "", regex=True)
df["card_distance_km"] = df["card_charact"].apply(
    lambda x: x.split(", ")[-1] if len(x.split(", ")) > 0 else "NaN"
)
df.loc[
    ~(
        ~(df["card_distance_km"].str.contains(r"^\D+$"))
        & (~(df["card_distance_km"].str.contains(r"[а-яА-Яa-zA-Z]")))
    ),
    "card_distance_km",
] = -10
df["card_distance_km"] = (
    df["card_distance_km"].str.replace(" ", "").fillna(-10).astype("int")
)

# vehicleEngine_engineDisplacement
df["vehicleEngine_engineDisplacement"] = (
    df["vehicleEngine_engineDisplacement"].str.replace(" LTR", "").astype("float")
)

# card_engine
df["card_engine"] = df["card_engine"].str.replace("дизель", "NaN")
df["card_engine"] = (
    df["card_engine"]
    .str.replace(" л.с.", "hp")
    .str.replace(" ", "")
    .str.replace("л\(", "lit_")
    .str.replace("\)", "")
    .str.replace("л", "lit")
)

df.loc[
    (~df["card_engine"].str.contains("hp")) & (df["card_engine"].str.contains("lit")),
    "card_lit",
] = df["card_engine"]

df.loc[
    (df["card_engine"].str.contains("hp")) & (~df["card_engine"].str.contains("lit")),
    "card_hp",
] = df["card_engine"]

df.loc[
    (df["card_engine"].str.contains("hp")) & (df["card_engine"].str.contains("lit")),
    "card_lit",
] = df["card_engine"].str.split("_", expand=True)[0]
df.loc[
    (df["card_engine"].str.contains("hp")) & (df["card_engine"].str.contains("lit")),
    "card_hp",
] = df["card_engine"].str.split("_", expand=True)[1]

df["card_lit"] = df["card_lit"].fillna("-10").str.replace("lit", "").astype("float")
df["card_hp"] = df["card_hp"].fillna("-10").str.replace("hp", "").astype("int")

# fill vehicleEngine_enginePower and vehicleEngine_engineDisplacement
df["vehicleEngine_enginePower"] = (
    df["vehicleEngine_enginePower"]
    .str.replace(" N12", "")
    .fillna(df["card_hp"])
    .astype("int")
)
df["vehicleEngine_engineDisplacement"] = df["vehicleEngine_engineDisplacement"].fillna(
    df["card_lit"]
)

# fuelType column has no hybrid category
df.loc[df["card_charact"].str.contains("гибрид"), "fuelType"] = "hybrid"

# drop processed columns
df = df.drop(
    [
        "offers_url",
        "card_charact",
        "card_engine",
        "card_lit",
        "card_hp",
        "vehicleEngine_name",
        # 'offer_lowPrice',
        # 'offer_highPrice',
        # 'offer_offerCount',
        "modelDate",
        "numberOfDoors",
        "vehicleConfiguration",
        # 'boundingbox'
    ],
    axis=1,
).reset_index(drop=True)

for col in df.select_dtypes(include="object").columns:
    if len(df[col].unique()) < 35:
        print(col, ":", df[col].unique())

bodyType : ['Хэтчбек 5 дв.' 'Открытый' 'Купе' 'Минивэн' 'Джип 5 дв.' 'Пикап'
 'Хэтчбек 3 дв.' 'Седан' 'Универсал' 'Джип 3 дв.' 'Лифтбек']
color : ['Черный' 'Серый' 'Синий' 'Бордовый' 'Зеленый' 'Серебристый' 'Голубой' nan
 'Белый' 'Красный' 'Оранжевый' 'Розовый' 'Золотистый' 'Фиолетовый'
 'Бежевый' 'Желтый' 'Коричневый']
fuelType : ['Бензин' 'Дизель' 'hybrid' nan 'Электро']
vehicleTransmission : ['АКПП' 'Вариатор' 'Автомат' 'Механика' nan 'Робот']
card_price_mark : ['без_оценки' nan 'нормальная_цена' 'хорошая_цена' 'отличная_цена'
 'высокая_цена']
card_repair : [nan 'repaired']
card_noDocs : [nan 'noDocs']
card_driveType : ['передний' 'задний' '4WD' nan]
flag_new : ['old' 'new']


In [5]:
# Fill the rest of columns

#
# color
df["color"].loc[df["color"].isna()] = "no_color"

#
# offers_price (fill the discounted values)
df.loc[~df["card_price_disc"].isna(), "card_price_mark"] = "NaN"
df.loc[~df["card_price_disc"].isna(), "offers_price"] = df[
    "card_price_disc"
].str.replace("\xa0", "")
df["offers_price"] = df["offers_price"].astype("int")

#
# card_price_mark
df["card_price_mark"] = (
    df["card_price_mark"].fillna("без_оценки").replace("NaN", "без_оценки")
)

#
# fuelType
intermediate_fueltype = (
    df.loc[(~df.fuelType.isna())]
    .groupby(["brand", "model"])["fuelType"]
    .agg(lambda x: x.mode()[0])
    .reset_index()
)
df = df.merge(
    intermediate_fueltype,
    on=["brand", "model"],
    how="left",
    suffixes=("", "_intermediate"),
)

df["fuelType"] = df["fuelType"].fillna(df["fuelType_intermediate"])
df = df.drop("fuelType_intermediate", axis=1)

# fill the rest with mode category
df["fuelType"] = df["fuelType"].fillna(df["fuelType"].mode()[0])

#
# vehicleTransmission
intermediate_vehicleTransmission = (
    df.loc[(~df.vehicleTransmission.isna())]
    .groupby(["brand", "model"])["vehicleTransmission"]
    .agg(lambda x: x.mode()[0])
    .reset_index()
)

df = df.merge(
    intermediate_vehicleTransmission,
    on=["brand", "model"],
    how="left",
    suffixes=("", "_intermediate"),
)

df["vehicleTransmission"] = df["vehicleTransmission"].fillna(
    df["vehicleTransmission_intermediate"]
)
df = df.drop("vehicleTransmission_intermediate", axis=1)

# fill the rest with mode category
df["vehicleTransmission"] = df["vehicleTransmission"].fillna(
    df["vehicleTransmission"].mode()[0]
)

#
# card_driveType
intermediate_card_driveType = (
    df.loc[(~df.card_driveType.isna())]
    .groupby(["brand", "model"])["card_driveType"]
    .agg(lambda x: x.mode()[0])
    .reset_index()
)

df = df.merge(
    intermediate_card_driveType,
    on=["brand", "model"],
    how="left",
    suffixes=("", "_intermediate"),
)

df["card_driveType"] = df["card_driveType"].fillna(df["card_driveType_intermediate"])
df = df.drop("card_driveType_intermediate", axis=1)

# fill the rest with mode category
df["card_driveType"] = df["card_driveType"].fillna(df["card_driveType"].mode()[0])

#
# drop processed columns
df = df.drop(
    [
        "card_price_disc",
    ],
    axis=1,
).reset_index(drop=True)

# results
print("NaN values:")
print(df.isna().sum(), "\n")

for col in df.select_dtypes(exclude="object").columns:
    print(col, ":", sum(df[col] < 0))

NaN values:
bodyType                                 0
color                                    0
fuelType                                 0
productionDate                           0
vehicleTransmission                      0
vehicleEngine_engineDisplacement         0
vehicleEngine_enginePower                0
offers_price                             0
card_price_mark                          0
card_repair                         365864
card_noDocs                         368965
region                                   0
brand                                    0
model                                    0
car_id                                   0
card_driveType                           0
flag_new                                 0
card_distance_km                         0
dtype: int64 

productionDate : 0
vehicleEngine_engineDisplacement : 4001
vehicleEngine_enginePower : 1330
offers_price : 0
car_id : 0
card_distance_km : 65585


In [6]:
# lower string type all object columns
for col in df.columns:
    if col in df.select_dtypes(include="object"):
        df[col] = df[col].str.lower()

# translation
df["bodyType"] = df["bodyType"].replace(
    {
        "джип 5 дв.": "suv",
        "седан": "sedan",
        "хэтчбек 3 дв.": "hatchback_3d",
        "лифтбек": "liftback",
        "хэтчбек 5 дв.": "hatchback",
        "универсал": "wagon",
        "купе": "coupe",
        "минивэн": "van",
        "джип 3 дв.": "suv_3d",
        "открытый": "open",
        "пикап": "pickup",
    }
)

df["model"] = df["model"].replace(
    {
        "cee~d.": "ceed",
        "pro_cee~d": "pro_ceed",
    }
)

df["color"] = (
    df["color"]
    .replace(
        {
            "черный": "black",
            "белый": "white",
            "серебристый": "silver",
            "серый": "grey",
            "коричневый": "brown",
            "синий": "blue",
            "зеленый": "green",
            "бежевый": "beige",
            "бордовый": "burgundy",
            "красный": "red",
            "золотистый": "golden",
            "желтый": "yellow",
            "фиолетовый": "purple",
            "голубой": "blue",
            "оранжевый": "orange",
            "розовый": "pink",
            "no_color": "no_color",
        }
    )
    .fillna("no_color")
)

df["fuelType"] = (
    df["fuelType"]
    .replace(
        {
            "бензин": "gasoline",
            "hybrid": "hybrid",
            "дизель": "diesel",
            "электро": "electric",
        }
    )
    .fillna("no_fuelType")
)

df["vehicleTransmission"] = (
    df["vehicleTransmission"]
    .replace(
        {
            "акпп": "automatic",
            "автомат": "automatic",
            "механика": "mechaniс",
            "вариатор": "variator",
            "робот": "robot",
        }
    )
    .fillna("no_trans")
)

df["card_driveType"] = (
    df["card_driveType"]
    .replace(
        {
            "передний": "front",
            "4wd": "awd",
            "задний": "rear",
        }
    )
    .fillna("no_driveType")
)

df["card_price_mark"] = (
    df["card_price_mark"]
    .replace(
        {
            "хорошая_цена": "good",
            "отличная_цена": "excellent",
            "нормальная_цена": "normal",
            "без_оценки": "no_price",
            "высокая_цена": "high",
        }
    )
    .fillna("no_price")
)

df["card_repair"] = (
    df["card_repair"].replace({"repaired": "repaired"}).fillna("not_repaired")
)

df["card_noDocs"] = df["card_noDocs"].replace({"nodocs": "have_docs"}).fillna("no_docs")

# df['card_from_owner'] = df['card_from_owner'].replace({
#     'от_собственника': 'owner'
# }).fillna('no_owner')

# df['card_probegCat'] = df['card_probegCat'].replace({
#     'новый': 'new',
#     'без_пробега_по_рф': 'foreign'
# }).fillna('used')

df[:5]

,bodyType,color,fuelType,productionDate,vehicleTransmission,vehicleEngine_engineDisplacement,vehicleEngine_enginePower,offers_price,card_price_mark,card_repair,card_noDocs,region,brand,model,car_id,card_driveType,flag_new,card_distance_km
0,hatchback,black,gasoline,2006,automatic,1.50,109,615000,no_price,not_repaired,no_docs,nakhodka,daihatsu,coo,53197435,front,old,240000
1,hatchback,grey,gasoline,2006,automatic,1.50,109,760000,no_price,not_repaired,no_docs,ussuriisk,daihatsu,coo,816310958,front,old,180000
2,hatchback,blue,gasoline,2006,automatic,1.30,92,595000,no_price,not_repaired,no_docs,berdsk,daihatsu,coo,619957730,front,old,180000
3,hatchback,burgundy,gasoline,2007,automatic,1.50,109,800000,no_price,not_repaired,no_docs,kemerovo,daihatsu,coo,54031033,front,old,200000
4,hatchback,green,gasoline,2008,automatic,1.50,109,630000,no_price,not_repaired,no_docs,lesozavodsk,daihatsu,coo,53977162,front,old,240000


In [7]:
# Clean dublicated on car id's

print(
    f"Dropped data that was parsed multiple times: {df.duplicated().sum()} / {df.shape[0]}"
)
df = df.drop_duplicates(subset="car_id").reset_index(drop=True)

Dropped data that was parsed multiple times: 3481 / 373518


In [8]:
# Check NaN numeric data

for col in df.select_dtypes(exclude="object").columns:
    print(col, ":", sum(df[col] < 0))

productionDate : 0
vehicleEngine_engineDisplacement : 3999
vehicleEngine_enginePower : 1317
offers_price : 0
car_id : 0
card_distance_km : 64244


In [9]:
# Try fill first two NaN (-10) columns

# card_distance_km
df.loc[(df.card_distance_km < 0) & (df.flag_new == "new"), "card_distance_km"] = 0

# vehicleEngine_engineDisplacement
engineDisplacement_mean = (
    df.loc[df.vehicleEngine_engineDisplacement > 0]
    .groupby(["brand", "model"])["vehicleEngine_engineDisplacement"]
    .mean()
    .round(1)
)

df = df.merge(
    engineDisplacement_mean,
    on=["brand", "model"],
    how="left",
    suffixes=("", "_intermediate"),
)

# check rows that has non-zero values
selected_rows = df.loc[
    (df["vehicleEngine_engineDisplacement"] < 0)
    & (~df["vehicleEngine_engineDisplacement_intermediate"].isna()),
    [
        "brand",
        "model",
        "vehicleEngine_engineDisplacement",
        "vehicleEngine_engineDisplacement_intermediate",
    ],
]

# Fill column 'vehicleEngine_engineDisplacement' with values that are non-zero in 'vehicleEngine_engineDisplacement_intermediate'
df.loc[
    (df["vehicleEngine_engineDisplacement"] < 0)
    & (~df["vehicleEngine_engineDisplacement_intermediate"].isna()),
    "vehicleEngine_engineDisplacement",
] = selected_rows["vehicleEngine_engineDisplacement_intermediate"]

# Drop intermediate column
df.drop("vehicleEngine_engineDisplacement_intermediate", axis=1, inplace=True)

#
# vehicleEngine_enginePower
vehicleEngine_enginePower_mean = (
    df.loc[df.vehicleEngine_enginePower > 0]
    .groupby(["brand", "model"])["vehicleEngine_enginePower"]
    .mean()
    .round(0)
)

df = df.merge(
    vehicleEngine_enginePower_mean,
    on=["brand", "model"],
    how="left",
    suffixes=("", "_intermediate"),
)

# check rows that has non-zero values
selected_rows = df.loc[
    (df["vehicleEngine_enginePower"] < 0)
    & (~df["vehicleEngine_enginePower_intermediate"].isna()),
    [
        "brand",
        "model",
        "vehicleEngine_enginePower",
        "vehicleEngine_enginePower_intermediate",
    ],
]

# Fill column 'vehicleEngine_engineDisplacement' with values that are non-zero in 'vehicleEngine_engineDisplacement_intermediate'
df.loc[
    (df["vehicleEngine_enginePower"] < 0)
    & (~df["vehicleEngine_enginePower_intermediate"].isna()),
    "vehicleEngine_enginePower",
] = selected_rows["vehicleEngine_enginePower_intermediate"].astype("int")

# Drop intermediate column
df.drop("vehicleEngine_enginePower_intermediate", axis=1, inplace=True)

#
# Check NaN data after transformation
sum_ = 0
for col in df.columns:
    if col in df.select_dtypes(exclude="object").columns:
        if sum(df[col] < 0) > 0:
            print(col, ":", sum(df[col] < 0))
        sum_ += sum(df[col] < 0)

print("\n", "Total NaN's: ", sum_, sep="")

vehicleEngine_engineDisplacement : 3252
vehicleEngine_enginePower : 5
card_distance_km : 4878

Total NaN's: 8135


### Regions preprocessing

**Methodology:**

- Obtain a list of unique regions and add N links in each key for parsing
- We feed the links to the parser and get the regions from the site drom.ru (the parser is presented in a separate file)
- We get a list of all regions in the format location_en - location_ru
- Feeding the location_ru column to the Nominatim api (some cells have been preprocessed so that Nominatim finds the locations correctly)
- Сombine the received data with a new data frame
- We use the obtained coordinates and the exact unified names of regions and regions in order to build a heatmap based on them using the plotly library and perform additional geolocation analysis

In [10]:
# from geopy.geocoders import Nominatim


# def get_location(address):
#     geolocator = Nominatim(user_agent="my_app")
#     try:
#         location = geolocator.geocode(address,
#                                       country_codes=["ru", 'by', 'kz', 'ua'],
#                                       addressdetails=True).raw
#     except:
#         location = None

#     return location


# def get_location(address, accepted_address_types=None):
# accepted_address_types = [
#     'country', 'state', 'county', 'city', 'town', 'borough', 'village',
#     'suburb', 'neighbourhood', 'settlement', 'industrial'
# ]
#     geolocator = Nominatim(user_agent="my_app")
#     try:
#         location = geolocator.geocode(address, country_codes="ru", addressdetails=True).raw
#         if location is not None:
#             address_type = location.get('address', {}).get('type')
#             if address_type in accepted_address_types:
#                 return location
#     except:
#         pass

#     return None


# def write_txt(file_path, list):
#     with open(file_path, "w") as f:
#         f.write('\n'.join(list))


# def open_txt(file_path):
#     with open(file_path) as f:
#         data = [
#             'Российская федерация, ' + line.strip() for line in f.readlines()
#         ]
#     return data


# # Get addresses
# srch_list = df.region.unique().tolist()

# data = []
# for srch in srch_list:
#     srch_ = 'Russian Federation, ' + srch
#     try:
#         location_raw = get_location(srch_)

#         addresstype = location_raw['addresstype']
#         name = location_raw['name']
#         display_name = location_raw['display_name']
#         latitude = location_raw['lat']
#         longtitude = location_raw['lon']
#         boundingbox = location_raw['boundingbox']

#         # Create a dictionary with the data for each row
#         row_data = {
#             'srch': srch,
#             'addresstype': addresstype,
#             'name': name,
#             'display_name': display_name,
#             'latitude': latitude,
#             'longitude': longtitude,
#             'boundingbox': boundingbox
#         }
#         data.append(row_data)

#     except:
#         pass

# # create df
# unique_location_df = pd.DataFrame(data)
# unique_location_df.to_csv('unique_location_df.csv')
# unique_location_df

# # # Merge with the df_old data frame on the "srch" column
# # df_merged = pd.merge(df_old,
# #                      unique_location_df,
# #                      left_on='region',
# #                      right_on='srch',
# #                      how='left')

In [11]:
# unique_regions = sorted(df.region.unique().tolist())

# data = []
# for reg in unique_regions:
#     links = df.loc[df.region == reg, 'offers_url'].tolist()[:5]
#     for link in links:
#         data.append({
#             'region_en': reg,
#             'link': link
#         })

# scrum_regions = pd.DataFrame(data)
# # scrum_regions.to_csv('regions_df.csv', index=False)

In [12]:
# from geopy.geocoders import Nominatim


# def get_location(address):
#     geolocator = Nominatim(user_agent="my_app")
#     try:
#         location = geolocator.geocode(address, country_codes=["ru", 'by', 'kz', 'ua'], addressdetails=True).raw
#     except:
#         location = None
#     return location


# #
# region_dict_df = pd.read_excel('region_dict_df.xlsx')
# region_dict_df = region_dict_df[3035:]
# for index, row in region_dict_df.iterrows():
#     location_raw = get_location(row['location_ru'])
#     try:
#         region_dict_df.at[index, 'lat'] = location_raw['lat']
#         region_dict_df.at[index, 'lon'] = location_raw['lon']
#         bounding_box_str = ', '.join(location_raw['boundingbox'])
#         region_dict_df.at[index, 'boundingbox'] = bounding_box_str
#         region_dict_df.at[index, 'addresstype'] = location_raw['addresstype']
#         region_dict_df.at[index, 'name'] = location_raw['name']
#         try:
#             region_dict_df.at[index, 'state'] = location_raw['address']['state']
#         except:
#             region_dict_df.at[index, 'state'] = np.nan
#         try:
#             region_dict_df.at[index, 'region'] = location_raw['address']['region']
#         except:
#             region_dict_df.at[index, 'region'] = np.nan
#         region_dict_df.at[index, 'country_code'] = location_raw['address']['country_code']
#     except:
#         pass

# # region_dict_df
# # region_dict_df.to_excel('region_dict_df_.xlsx', index=False)

In [13]:
# credits: https://stackoverflow.com/questions/66833670/geopandas-map-centering-with-countries

In [14]:
# import geopandas as gpd


# def plot_auto_geo(data: pd.DataFrame,
#                   geo_data: gpd.GeoDataFrame,
#                   title: str,
#                   cmap: str = 'Spectral_r',
#                   agg_col: str = 'log_price',
#                   func: str = 'mean',
#                   group_col: str = 'region',
#                   ax=None):
#     """Отображение на карте агрегированных данных"""

#     agg_data = data.groupby(group_col)[agg_col].agg(func)
#     auto_geo = gpd.GeoDataFrame(
#         pd.merge(agg_data, geo_data, on=group_col, how='left'))

#     mo_gdf_wm = auto_geo.to_crs({'init': 'epsg:4326'})
#     mo_gdf_wm.plot(column=agg_col,
#                    linewidth=0.5,
#                    cmap=cmap,
#                    legend=True,
#                    figsize=(20, 71),
#                    legend_kwds={
#                        'label': func,
#                        'orientation': "vertical"
#                    },
#                    edgecolor="white",
#                    vmin=agg_data.min(),
#                    vmax=agg_data.max(),
#                    ax=ax,
#                    aspect='auto')

#     if ax:
#         ax.set_title(title)
#         plt.tight_layout()
#     else:
#         plt.title(title, fontdict={'fontsize': 16})
#         plt.axis('off')
#         plt.show()

In [15]:
# def shift_geom(shift, row):
#     shift = 180
#     moved_map = []
#     splitted_map = []
#     border = LineString([(shift, 90), (shift, -90)])

#     splitted_map = split(row, border)
#     items = list(splitted_map)

#     for item in items:
#         minx, miny, maxx, maxy = item.bounds
#         if minx >= shift:
#             moved_map.append(translate(item, xoff=-180 - shift))
#         else:
#             moved_map.append(translate(item, xoff=180 - shift))

#     gdf = gpd.GeoDataFrame({"geometry": moved_map})
#     return gdf.geometry.unary_union

In [16]:
# fig, axes = plt.subplots(ncols=2, figsize=(20, 6))

# plot_auto_geo(data=df_old,
#               geo_data=russia_salary_geo,
#               title='Median price for new vehicles',
#               cmap=colors[0],
#               agg_col='log_price',
#               func='median',
#               group_col='region',
#               ax=axes[0])

# mo_gdf_wm = russia_salary_geo.to_crs({'init': 'epsg:4326'})

# mo_gdf_wm.plot(column='median_salary',
#                linewidth=0.5,
#                cmap='Reds',
#                legend_kwds={
#                    'label': "median salary",
#                    'orientation': "vertical"
#                },
#                edgecolor="white",
#                ax=axes[1],
#                aspect='auto')

# axes[1].set_title('Средняя зарплата по стране', fontdict={'fontsize': 14})
# plt.show()

In [18]:
# final stage of region data preparation
region_dict_df = pd.read_excel("data/region_dict_df_.xlsx")
df = df.merge(
    region_dict_df,
    how="left",
    left_on="region",
    right_on="location_en",
    suffixes=("", "_nom"),
)
df.shape

(369749, 28)

In [19]:
# Ukraine preprocession (no personal preference)
df.loc[(df.location_en == "sevastopol"), ["region_nom", "country_code"]] = [
    "Южный федеральный округ",
    "ru",
]
df["state"] = df["state"].replace("Автономна Республіка Крим", "Республика Крым")
df.loc[(df.location_en == "simferopol"), ["state"]] = ["Симферополь"]

# processing new columns
df.loc[
    (df["country_code"].isna()) | (df["state"] == "Севастополь"),
    ["region_nom", "country_code"],
] = ["Южный федеральный округ", "ru"]
df[["state", "region_nom"]] = df[["state", "region_nom"]].fillna("no_info")
df.loc[
    (df.state == "Республика Крым")
    | (df.state == "Симферополь")
    | (df.state == "Симферополь"),
    ["region_nom", "country_code"],
] = ["Южный федеральный округ", "ru"]

# delete columns that contain similar information or might cause overfitting in predictive analysis
df.drop(["location_ru", "location_en", "boundingbox"], axis=1, inplace=True)

# rename regions
df["region_nom"] = df["region_nom"].replace(
    {
        "Сибирский федеральный округ": "сфо",
        "Дальневосточный федеральный округ": "дфо",
        "Южный федеральный округ": "юфо",
        "Северо-Западный федеральный округ": "сзфо",
        "Приволжский федеральный округ": "пфо",
        "Центральный федеральный округ": "цфо",
        "Уральский федеральный округ": "уфо",
        "Северо-Кавказский федеральный округ": "скфо",
    }
)

df.isna().sum()

bodyType                            0
color                               0
fuelType                            0
productionDate                      0
vehicleTransmission                 0
vehicleEngine_engineDisplacement    0
vehicleEngine_enginePower           0
offers_price                        0
card_price_mark                     0
card_repair                         0
card_noDocs                         0
region                              0
brand                               0
model                               0
car_id                              0
card_driveType                      0
flag_new                            0
card_distance_km                    0
addresstype                         0
name                                0
state                               0
region_nom                          0
lat                                 0
lon                                 0
country_code                        0
dtype: int64

### Check anomaly and outliers

In [20]:
# # categorize columns
# target_col = ['offers_price']
# df[['car_id', 'productionDate']] = df[['car_id', 'productionDate']].astype('object')
# cat_cols = df.select_dtypes(include=['object', 'category']).columns.tolist()
# num_cols = df.select_dtypes(exclude=['object', 'category']).columns.tolist()

In [21]:
# delete outliers in certain numeric columns:
num_rows = df.shape[0]

low_q = 0.005
high_q = 0.995

# extremes
df = df.loc[
    ~(
        (df["card_distance_km"] > df["card_distance_km"].quantile(high_q))
        | (df["productionDate"] < df["productionDate"].quantile(low_q))
        | (
            df["vehicleEngine_enginePower"]
            > df["vehicleEngine_enginePower"].quantile(high_q)
        )
    )
].reset_index(drop=True)

# general
# df = df[~((df < (Q1 - 1.5 * IQR)) | (df > (Q3 + 1.5 * IQR))).any(axis=1)]

outliers_elim = num_rows - df.shape[0]
print(
    "Rows with outliers eliminated:",
    outliers_elim,
    f"~ {round((outliers_elim / num_rows * 100), 2)}%",
)

Rows with outliers eliminated: 5314 ~ 1.44%


In [22]:
# delete unpopular models
models_count = df.groupby(["model"])["car_id"].count()
drop_models = models_count[models_count <= 15].index
df = df[~df.model.isin(drop_models)].reset_index(drop=True)

### target normalization, separate data frames

In [23]:
# price logged
"""
By normalizing the target variable, we preserve the distribution pattern
for plotting in exploratory analysis. Outliers are smoothed out.
"""
df["log_price"] = df["offers_price"].apply(lambda x: np.log1p(x))

In [24]:
# add region related metrics
merged_df = df.copy()
merged_df = df.merge(gos_metrics, how="left", left_on="state", right_on="key_")
merged_df = merged_df.drop("key_", axis=1)

# fill absent columns
new_cols = [
    "income_per_capita",
    "income_median",
    "cost_living",
    "population",
    "retiree",
    "retiree_perc",
]

for col in new_cols:
    gos_metrics.loc[gos_metrics.key_ == "Севастополь", [col]] = [
        gos_metrics.loc[gos_metrics.key_ == "Республика Крым", col]
    ]
merged_df[new_cols] = merged_df[new_cols].fillna(-10)

# lower string type all object columns
for col in ["name", "state"]:
    merged_df[col] = merged_df[col].str.lower()

df[:4]

,bodyType,color,fuelType,productionDate,vehicleTransmission,vehicleEngine_engineDisplacement,vehicleEngine_enginePower,offers_price,card_price_mark,card_repair,card_noDocs,region,brand,model,car_id,card_driveType,flag_new,card_distance_km,addresstype,name,state,region_nom,lat,lon,country_code,log_price
0,open,silver,gasoline,2017,variator,0.70,64,1300000,no_price,not_repaired,no_docs,seversk,daihatsu,copen,51223045,front,old,49000,city,Северск,Томская область,сфо,56.60,84.88,ru,14.08
1,open,grey,gasoline,2007,automatic,0.70,64,925000,no_price,not_repaired,no_docs,novosibirsk,daihatsu,copen,587675409,front,old,154578,city,городской округ Новосибирск,Новосибирская область,сфо,55.03,82.92,ru,13.74
2,open,blue,gasoline,2019,variator,0.70,64,1650000,no_price,not_repaired,no_docs,yalta,daihatsu,copen,54144789,front,old,134000,town,Ялта,Республика Крым,юфо,44.50,34.16,ru,14.32
3,open,no_color,gasoline,2019,variator,0.70,64,1350000,no_price,not_repaired,no_docs,vladivostok,daihatsu,copen,53332082,front,old,12000,city,Владивосток,Приморский край,дфо,43.12,131.89,ru,14.12


In [ ]:
# save merged_df
# merged_df.to_csv('data/autodrom_merged_df.csv', sep=';', index=False)